# Precipitation measurements

Example of homemade rain gauge precipitation measurements with the help of kids toy-buckets and some bayesian mathematics.

See https://en.wikipedia.org/wiki/Rain_gauge

Measurements were done overnight between 24.-25.7.2020.

## Weather station measurements
Espoo, Nuuksio, <15 km from the measurement location  
Espoo, Tapiola, <15 km from the measurement location  
https://www.ilmatieteenlaitos.fi/havaintojen-lataus

## Problem definition

The aim is to estimate precipitation based on simple measurement setup.
Two buckets (blue and yellow) were left outside overnight to gather rain precipitation.
The diameter of the bucket was measured a few times with a tape measure.
At the morning next day, the buckets were weighted with and without the precipitated water.
Scale was tared between the measurements and the buckets were placed approximately center.

Scale used for the weighting has approximately absolute accuracy of +/- 2 grams and the result is rounded to the closest 1 gram.
Tape measure results were inferred from the photograph and are given +/- 2 centimeter accuracy.

In [1]:
import numpy as np
import pandas as pd

## FMI Measurement data

In [2]:
fmi_data = pd.read_csv("./FMI_DATA/precipitation.csv", index_col=0)
fmi_data

,Precipitation overnight (mm)
Nuuksio,1.350000
Tapiola,4.066667


# Data

In [3]:
import pickle

In [4]:
with open("DATA/measurement_data.pickle", "rb") as f:
    data = pickle.load(f)

In [5]:
data.keys()

dict_keys(['weight_bucket', 'weight_bucket_water', 'diameter', 'radius', 'wedge_outer', 'wedge_inner', 'wedge_length', 'density_water'])

# Engineering approximation with uncertainty --> normal models it is

In [6]:
from upandas import separate_to_u, u_to_separate

In [7]:
def add_uncertainty(df, ufunc, inplace=True):
    udf = df.applymap(ufunc).rename(columns={key : f"u_{key}" for key in df.columns})
    if inplace:
        df[udf.columns] = udf
        return
    return udf

In [8]:
# Let's add error for measurements
# weights --> sqrt(2)
# distances --> sqrt(2)

for key in ['weight_bucket', 'weight_bucket_water']:
    add_uncertainty(data[key], lambda x: 2**0.5)
    

for key in ['diameter', 'wedge_outer', 'wedge_inner', 'wedge_length']:
    add_uncertainty(data[key], lambda x: 2**0.5)

In [9]:
udata = {key : separate_to_u(data[key]) for key in ['weight_bucket', 'weight_bucket_water', 'diameter', 'wedge_outer', 'wedge_inner', 'wedge_length']}

In [10]:
udata["radius"] = udata["diameter"] / 2

In [11]:
from helper_functions import calculate_bucket_area_uncertain

In [12]:
area = calculate_bucket_area_uncertain(
    udata["radius"].apply(lambda x: sum(x)/len(x)),
    udata["wedge_inner"].apply(lambda x: sum(x)/len(x)),
    udata["wedge_outer"].apply(lambda x: sum(x)/len(x)),
    udata["wedge_length"].apply(lambda x: sum(x)/len(x)),
)
area

blue      0.021+/-0.004
yellow    0.021+/-0.004
dtype: object

In [13]:
weight_water = udata["weight_bucket_water"].apply(lambda x: sum(x)/len(x)) - udata["weight_bucket"].apply(lambda x: sum(x)/len(x))
weight_water

blue      41.7+/-1.2
yellow    39.7+/-1.2
dtype: object

In [14]:
# (grams/1000) / density_water * 1000 --> grams / density_water
volume_water_litre = weight_water / data["density_water"]
volume_water_litre

blue      0.0417+/-0.0012
yellow    0.0397+/-0.0012
dtype: object

In [15]:
precipitation = volume_water_litre / area
precipitation

blue        2.0+/-0.4
yellow    1.86+/-0.34
dtype: object

In [16]:
mean_precipitation =  precipitation.values.sum() / len(precipitation.values)

In [17]:
print(f"Precipitation: {mean_precipitation:.2f} mm")

Precipitation: 1.93+/-0.25 mm


In [18]:
from uncertainties import correlation_matrix, covariance_matrix

In [19]:
names = ["mean_precipitation", *[f"weight_water {k}" for k in weight_water.index], *[f"area {k}" for k in area.index]]
pd.DataFrame(correlation_matrix([mean_precipitation, *list(weight_water.values), *list(area)]), index=names, columns=names)

,mean_precipitation,weight_water blue,weight_water yellow,area blue,area yellow
mean_precipitation,1.000000,0.110223,0.107607,-0.727809,-0.668256
weight_water blue,0.110223,1.000000,0.000000,0.000000,0.000000
weight_water yellow,0.107607,0.000000,1.000000,0.000000,0.000000
area blue,-0.727809,0.000000,0.000000,1.000000,0.000000
area yellow,-0.668256,0.000000,0.000000,0.000000,1.000000


In [20]:
names = ["mean_precipitation", *[f"weight_water {k}" for k in weight_water.index], *[f"area {k}" for k in area.index]]
pd.DataFrame(covariance_matrix([mean_precipitation, *list(weight_water.values), *list(area)]), index=names, columns=names)

,mean_precipitation,weight_water blue,weight_water yellow,area blue,area yellow
mean_precipitation,0.063336,0.032031,0.031271,-0.000698,-0.000649
weight_water blue,0.032031,1.333333,0.000000,0.000000,0.000000
weight_water yellow,0.031271,0.000000,1.333333,0.000000,0.000000
area blue,-0.000698,0.000000,0.000000,0.000015,0.000000
area yellow,-0.000649,0.000000,0.000000,0.000000,0.000015
